In [1]:
import numpy as np 
import pandas as pd 
from keras.preprocessing.text import Tokenizer
import keras 

In [2]:
# source text
# data = """ Jack and Jill went up the hill\n
# 		To fetch a pail of water\n
# 		Jack fell down and broke his crown\n
# 		And Jill came tumbling after\n """

path1 = "../Kebijakan Pemerintah/3.menetapkan.txt"
data = open(path1, "r").read()
data

'PERATURAN PRESIDEN TENTANG PERUBAHAN KEDUA ATAS PERATURAN PRESIDEN NOMOR 99 TAHUN 2O2O TENTANG PENGADAAN VAKSIN DAN PELAKSANAAN VAKSINASI DALAM RANGKA PENANGGULANGAN PANDEMI CORONA VIRUS DISEASE 2019 (COVID-19).Ketentuan ayat (2) Pasal 11A Peraturan Presiden Nomor 99 Tahun 2O2O tentang Pengadaan Vaksin dan Pelaksanaan Vaksinasi Dalam Rangka Penanggulangan Pandemi Corona Virus Disease 2019 (COVID-19) (Lembaran Negara Republik Indonesia Tahun 2O2O Nomor 2271 sebagaimana telah diubah dengan Peraturan Presiden Nomor 14 Tahun 2O2I tentang Perubahan Atas Peraturan Presiden Nomor 99 Tahun 2O2O tentang Pengadaan Vaksin dan Pelaksanaan Vaksinasi Dalam Rangka Penanggulangan Pandemi Corona Virus Disease 2019 (COVID- 19) (Lembaran Negara Republik Indonesia Tahun 2O2l Nomor 66) diubah sehingga berbunyi sebagai berikut: Dalam hal pengadaan vaksin dilakukan melalui penugasan kepada badan usaha milik negara, penunjukan langsung kepada badan usaha penyedia, atau kerjasama lembaga/badan internasional y

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [4]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 161


In [5]:
# create word -> word sequences
sequences = []
step = 2
for i in range(step, len(encoded)):
	sequence = encoded[i-step:i+1]
	sequences.append(sequence)

sequences = np.array(sequences)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 406


In [6]:
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 3


In [7]:
sequences

array([[ 2,  7, 19],
       [ 7, 19, 61],
       [19, 61, 82],
       ...,
       [ 6, 41, 22],
       [41, 22, 42],
       [22, 42, 43]])

In [8]:
# split into X and y elements
x, y = sequences[:, :-1],sequences[:, -1]

In [9]:
y = pd.get_dummies(y).values

In [10]:
model = keras.Sequential([
            keras.layers.Embedding(output_dim=10, input_dim=vocab_size, input_length=max_length-1), 
            keras.layers.LSTM(50), 
            keras.layers.Dense(y.shape[1], activation="softmax")
        ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 10)             1610      
_________________________________________________________________
lstm (LSTM)                  (None, 50)                12200     
_________________________________________________________________
dense (Dense)                (None, 160)               8160      
Total params: 21,970
Trainable params: 21,970
Non-trainable params: 0
_________________________________________________________________


In [11]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(x, y, epochs=500, verbose=2)

Epoch 1/500
13/13 - 33s - loss: 5.0738 - accuracy: 0.0345
Epoch 2/500
13/13 - 0s - loss: 5.0668 - accuracy: 0.0394
Epoch 3/500
13/13 - 0s - loss: 5.0597 - accuracy: 0.0394
Epoch 4/500
13/13 - 0s - loss: 5.0502 - accuracy: 0.0369
Epoch 5/500
13/13 - 0s - loss: 5.0359 - accuracy: 0.0345
Epoch 6/500
13/13 - 0s - loss: 5.0149 - accuracy: 0.0345
Epoch 7/500
13/13 - 0s - loss: 4.9795 - accuracy: 0.0369
Epoch 8/500
13/13 - 0s - loss: 4.9209 - accuracy: 0.0345
Epoch 9/500
13/13 - 0s - loss: 4.8308 - accuracy: 0.0345
Epoch 10/500
13/13 - 0s - loss: 4.7235 - accuracy: 0.0320
Epoch 11/500
13/13 - 0s - loss: 4.6304 - accuracy: 0.0320
Epoch 12/500
13/13 - 0s - loss: 4.5690 - accuracy: 0.0320
Epoch 13/500
13/13 - 0s - loss: 4.5175 - accuracy: 0.0320
Epoch 14/500
13/13 - 0s - loss: 4.4645 - accuracy: 0.0394
Epoch 15/500
13/13 - 0s - loss: 4.4149 - accuracy: 0.0493
Epoch 16/500
13/13 - 0s - loss: 4.3682 - accuracy: 0.0616
Epoch 17/500
13/13 - 0s - loss: 4.3194 - accuracy: 0.0690
Epoch 18/500
13/13 - 0

In [12]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		# print("Iterasi ke-", _)
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# print("Endocded : ", encoded)
		# pre-pad sequences to a fixed length
		encoded = keras.preprocessing.sequence.pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0) + 1
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [13]:
# Menimbang 
# a. bahwa ketentuan
# b. bahwa berdasarkan
menimbang = ["peraturan presiden", "dalam hal", "pengambilalihan tanggung", "dalam hal", "agar setiap"]

for i in menimbang:
    print(i, "\nKalimat:")
    print(generate_seq(model, tokenizer, max_length-1, i, 50))
    print("\n\n")

peraturan presiden 
Kalimat:


C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


peraturan presiden nomor 99 tahun 2o2o tentang pengadaan vaksin dan pelaksanaan vaksinasi dalam rangka penanggulangan pandemi corona virus disease 2019 covid 19 sebagai bencana nasional sebagaimana dimaksud pada ayat 1 ayat 2 pasal 11a peraturan presiden nomor 99 tahun 2o2o tentang pengadaan vaksin dan pelaksanaan vaksinasi dalam rangka penanggulangan pandemi corona virus



dalam hal 
Kalimat:
dalam hal pada saat dicabutnya penetapan kedaruratan kesehatan masyarakat covid l9 dan penetapan bencana nonalam penyebaran covid 19 sebagai bencana nasional sebagaimana dimaksud pada ayat 1 ayat 2 pasal 11a peraturan presiden nomor 99 tahun 2o2o tentang pengadaan vaksin dan pelaksanaan vaksinasi dalam rangka penanggulangan pandemi corona virus disease 2019 covid 19



pengambilalihan tanggung 
Kalimat:
pengambilalihan tanggung jawab hukum sebagaimana dimaksud pada ayat 1 ayat 2 pasal 11a peraturan presiden nomor 99 tahun 2o2o tentang pengadaan vaksin dan pelaksanaan vaksinasi dalam rangka penan